In [1]:
import pandas as pd
import numpy as np

In [45]:
df = pd.read_csv('../data/learn_pandas.csv')

In [33]:
df[['School','Height','Transfer','Test_Number']].drop_duplicates(['School','Transfer']).reset_index(drop=True).set_index(['School', 'Transfer']).unstack()

Height               Test_Number          
Transfer                         NaN      N      Y         NaN    N    Y
School                                                                  
Fudan University               159.3    NaN  177.2         1.0  2.0  1.0
Peking University                NaN  166.5    NaN         1.0  1.0  1.0
Shanghai Jiao Tong University    NaN  158.9    NaN         2.0  1.0  NaN
Tsinghua University            162.1  158.0  157.4         1.0  1.0  1.0

In [66]:
%timeit pd.crosstab(df.School, df.Transfer, values=df.Height, aggfunc='skew')

8.7 ms ± 133 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [67]:
%timeit data.pivot_table(index='School', columns='Transfer', values='Height', aggfunc='skew')

7.52 ms ± 55.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [61]:
data = df[['School', 'Transfer', 'Height']]
data.pivot_table(index='School', columns='Transfer', values='Height', aggfunc='count')

Transfer,N,Y
School,,
Fudan University,32.0,1.0
Peking University,27.0,1.0
Shanghai Jiao Tong University,52.0,NaN
Tsinghua University,56.0,4.0


In [43]:
df = pd.DataFrame(np.ones((4,2)),
                     index = pd.Index([('A', 'cat', 'big'),
                                       ('A', 'dog', 'small'),
                                       ('B', 'cat', 'big'),
                                       ('B', 'dog', 'small')]),
                     columns=['col_1', 'col_2'])
my_index = df.index.to_list()
my_index[1] = my_index[0]
df.index = pd.Index(my_index)
df.T

A         B      
       cat       cat   dog
       big  big  big small
col_1  1.0  1.0  1.0   1.0
col_2  1.0  1.0  1.0   1.0

In [5]:
df = pd.read_csv('../data/learn_pandas.csv')

In [45]:
df = pd.DataFrame({'Class':[1,2],'Name':['San Zhang', 'Si Li'], 'Chinese':[80, 75], 'Math':[90, 85]})
df = df.melt(id_vars=['Class', 'Name'], value_vars=['Chinese', 'Math'], var_name='Subject', value_name='Grade')
df.pivot(index=['Class', 'Name'], columns='Subject', values='Grade').reset_index().rename_axis(columns={'Subject':''})

,Class,Name,Chinese,Math
0,1,San Zhang,80,90
1,2,Si Li,75,85


In [61]:
df = pd.DataFrame({'Class':[1,1,2,2],
                   'Name':['San Zhang', 'San Zhang', 'Si Li', 'Si Li'],
                   'Subject':['Chinese', 'Math', 'Chinese', 'Math'],
                   'Grade':[80, 75, 90, 85]})
df2 = df.pivot(index = 'Name', columns = 'Subject', values = 'Grade')
df2.dfi.export('df1.png')

In [28]:
df = pd.DataFrame({'Class':[1, 1, 2, 2, 1, 1, 2, 2],
                   'Name':['San Zhang', 'San Zhang', 'Si Li', 'Si Li',
                           'San Zhang', 'San Zhang', 'Si Li', 'Si Li'],
                   'Examination': ['Mid', 'Final', 'Mid', 'Final',
                                   'Mid', 'Final', 'Mid', 'Final'],
                   'Subject':['Chinese', 'Chinese', 'Chinese', 'Chinese',
                              'Math', 'Math', 'Math', 'Math'],
                   'Grade':[80, 75, 85, 65, 90, 85, 92, 88],
                   'rank':[10, 15, 21, 15, 20, 7, 6, 2]})
df2 = df.pivot(index = ['Class', 'Name'],
               columns = ['Subject','Examination'],
               values = ['Grade','rank'])
df2

Grade                     rank                 
Subject         Chinese       Math       Chinese       Math      
Examination         Mid Final  Mid Final     Mid Final  Mid Final
Class Name                                                       
1     San Zhang      80    75   90    85      10    15   20     7
2     Si Li          85    65   92    88      21    15    6     2

In [31]:
df2 = df.pivot(index = ['Class', 'Name'],
               columns = ['Subject','Examination'],
               values = ['Grade','rank'])
df2.columns = df2.columns.map(lambda x:'_'.join(x))
df2 = df2.reset_index()
df2 = pd.wide_to_long(df2, stubnames=['Grade', 'rank'],
                           i = ['Class', 'Name'],
                           j = 'Subject_Examination',
                           sep = '_',
                           suffix = '.+')
df2 = df2.reset_index()
df2[['Subject', 'Examination']] = df2.Subject_Examination.str.split('_', expand=True)
df2 = df2.drop(columns='Subject_Examination')
df2 = df[['Class', 'Name', 'Examination', 'Subject', 'Grade', 'rank']].sort_values('Subject')
df2

,Class,Name,Examination,Subject,Grade,rank
0,1,San Zhang,Mid,Chinese,80,10
1,1,San Zhang,Final,Chinese,75,15
2,2,Si Li,Mid,Chinese,85,21
3,2,Si Li,Final,Chinese,65,15
4,1,San Zhang,Mid,Math,90,20
5,1,San Zhang,Final,Math,85,7
6,2,Si Li,Mid,Math,92,6
7,2,Si Li,Final,Math,88,2


In [21]:
df2

,Class,Name,Subject_Examination,Grade,rank
0,1,San Zhang,Chinese_Mid,80,10
1,1,San Zhang,Chinese_Final,75,15
2,1,San Zhang,Math_Mid,90,20
3,1,San Zhang,Math_Final,85,7
4,2,Si Li,Chinese_Mid,85,21
5,2,Si Li,Chinese_Final,65,15
6,2,Si Li,Math_Mid,92,6
7,2,Si Li,Math_Final,88,2


In [3]:
df = pd.DataFrame({'Class':[1,2],
                   'Name':['San Zhang', 'Si Li'],
                   'Chinese':[80, 90],
                   'Math':[80, 75]})
df_melted = df.melt(id_vars = ['Class', 'Name'],
        value_vars = ['Chinese', 'Math'],
        var_name = 'Subject',
        value_name = 'Grade')

In [9]:
df_melted.pivot(index = ['Class', 'Name'], columns='Subject', values='Grade').reset_index().rename_axis(columns={'Subject':''})

SyntaxError: invalid syntax (<ipython-input-9-2e007a100d8a>, line 1)

In [1]:
import pandas as pd
import numpy as np
pd.DataFrame({'Class':[1,1,2,2],
                      'Name':['San Zhang','San Zhang','Si Li','Si Li'],
                      'Subject':['Chinese','Math','Chinese','Math'],
                      'Grade':[80,75,90,85]}).pivot(index=['Class','Name'], columns='Subject', values='Grade')

1.1.3


,Subject,Chinese,Math
Class,Name,,
1,San Zhang,80,75
2,Si Li,90,85


In [35]:
df = pd.DataFrame({'Class':[1,2],'Name':['San Zhang', 'Si Li'],
                   'Chinese_Mid':[80, 75], 'Math_Mid':[90, 85],
                   'Chinese_Final':[80, 75], 'Math_Final':[90, 85]})
pd.wide_to_long(df, stubnames=['Chinese', 'Math'], i = ['Class', 'Name'], j='Examination', sep='_', suffix='.+')

Chinese  Math
Class Name      Examination               
1     San Zhang Mid               80    90
                Final             80    90
2     Si Li     Mid               75    85
                Final             75    85

In [49]:
df = pd.DataFrame({'Class':[1,2],
                  'Name':['San Zhang', 'Si Li'],
                 'Grade_Chinese':[80, 90],
                 'Grade_Math':[80, 75]})
pd.wide_to_long(df, stubnames='Grade',
                    i = ['Class', 'Name'],
                    j = 'Subject',
                     sep = '_',
                     suffix = '.+').reset_index()

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,1,San Zhang,Math,80
2,2,Si Li,Chinese,90
3,2,Si Li,Math,75


In [34]:
#import dataframe_image as dfi
df2.dfi.export('df1.png')